# Excution time measurements


In [2]:
cycles = 2

# Pandas

In [7]:
import pandas as pd
import sys
import os
import timeit

In [4]:
data1 = 'output/data1.csv'
data2 = 'output/data2.csv'
biggest = 'output/data2.csv'



file_pathss = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv'
]


file_paths = [
    'output/data1.csv',
    'output/data2.csv',
    'output/data3.csv',
    'output/data4.csv',
    'output/data5.csv',
    'output/data6.csv',
    'output/data7.csv',
    'output/data8.csv',
    'output/data9.csv',
    'output/data10.csv',
    'output/data12.csv',
    'output/data13.csv',
    'output/data14.csv',
    'output/data15.csv',
]

bigger_files = [
    'output/data16.csv',
    'output/data17.csv',
    'output/data18.csv',
    'output/data19.csv',
    'output/data20.csv',
    'output/data21.csv',
    'output/data22.csv',
    'output/data23.csv',
    'output/data24.csv'
]


def average_time(myList):
    total_time_spark = 0
    for i in myList:
        total_time_spark += i
    avg_time = total_time_spark / cycles
    return avg_time

In [4]:
df1 = pd.read_csv(data1)


# Data Loading

In [127]:
def read_csv(x):
    df = pd.read_csv(x)
    return df

def read_time(dis):
    start_time = timeit.default_timer()
    dataframe = read_csv(dis)
    end_time = timeit.default_timer()
    return end_time - start_time


def measure_average_read_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        timings_pandas = [read_time(file_path) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_loading(paths):
    avg_times = measure_average_read_time(paths)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average read time for data_{i}: {avg_time:.5f} seconds")
        
run_loading(file_paths)


In [9]:
# Measuring lead time of bigger datasize - 6Gb - 15 GB
#run_loading(bigger_files)

# Filtering and Selection

In [8]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]

# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def filter_and_select(filter_it):
    return filter_it.loc[(filter_it["5.164162635803223"] >= 6) & (filter_it["5.164162635803223"] <= 7)]

# Measurement - Filter_csv
def filter_time(dis):
    start_time = timeit.default_timer()
    dataframe = filter_and_select(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [filter_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_filter(addresses):
    avg_times = measure_average_filter_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")
        
#run_filter(file_paths)


In [5]:
#run_filter(bigger_files)

# Data Transformation

<h1>Spark</h1>

In [5]:
import pyspark
import timeit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, count, udf
import timeit
from pyspark.sql.types import StringType
from pyspark.sql import functions as F

# Data Loading

In [6]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

def read_csv(x):
    df = spark.read.option('header', 'true').csv(x, inferSchema=True)
    return df

def read_time(dis):
    start_time = timeit.default_timer()
    dataframe = read_csv(dis)
    end_time = timeit.default_timer()
    return end_time - start_time


def measure_average_read_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        timings_spark = [read_time(file_path) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_loading_spark(addresses):
    avg_times = measure_average_read_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average read time for data_{i}: {avg_time:.5f} seconds")


23/06/11 20:53:43 WARN Utils: Your hostname, Merons-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
23/06/11 20:53:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/11 20:53:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/11 20:53:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
#run_loading_spark(file_paths)

In [3]:
#run_loading_spark(bigger_files)

# Filtering data

# Trying to filter without action

In [1]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

# We are filtering from a column "5.164162635803223" which has values between and including 6 and 7.
def spark_filter_and_select(df):
    return df.filter((df["`5.164162635803223`"] >= 6) & (df["`5.164162635803223`"] <= 7))

# Measurement - Filter_csv
def spark_filter_time(df):
    start_time = timeit.default_timer()
    dataframe = spark_filter_and_select(df)
    end_time = timeit.default_timer()
    return end_time - start_time


def spark_measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = spark.read.option('header', 'true').csv(file_path, inferSchema=True)
        timings_spark = [spark_filter_time(local_df) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_spark_filter(filesss):
    avg_times = spark_measure_average_filter_time(filesss)

    for i, avg_time in enumerate(avg_times, 1):
        if i >= 11:
            print(f"Average filter time for data-{i + 1}: {avg_time:.5f} seconds")
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")

        
        
        


In [2]:
#run_spark_filter(file_paths)

In [1]:
#run_spark_filter(bigger_files)

# Data Transformation

# Trying to filter with an actual action

In [11]:
from pyspark.sql import SparkSession
import timeit

spark = SparkSession.builder.getOrCreate()

# We are filtering from a column "5.164162635803223" which has values between and including 6 and 7.
def spark_filter_and_select(df):
    return df.filter((df["`5.164162635803223`"] >= 6) & (df["`5.164162635803223`"] <= 7)).count()

# Measurement - Filter_csv
def spark_filter_time(df):
    start_time = timeit.default_timer()
    dataframe = spark_filter_and_select(df)
    end_time = timeit.default_timer()
    return end_time - start_time


def spark_measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = spark.read.option('header', 'true').csv(file_path, inferSchema=True)
        timings_spark = [spark_filter_time(local_df) for _ in range(cycles)]
        avg_time_spark = average_time(timings_spark)
        avg_times.append(avg_time_spark)
    return avg_times

def run_spark_filter(filesss):
    avg_times = spark_measure_average_filter_time(filesss)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Average filter time for data-{i}: {avg_time:.5f} seconds")


In [21]:
#run_spark_filter(file_paths)

In [20]:
#run_spark_filter(bigger_files)

# And for that reason, we also doing the results for pandas with counting of rows included

In [14]:
test = pd.read_csv("output/data1.csv")

addresses = ["output/data1.csv"]

# we are filtering from a column "5.164162635803223" which are in values b/n and including 6 and 7.
def filter_and_select(filter_it):
    return (filter_it.loc[(filter_it["5.164162635803223"] >= 6) & (filter_it["5.164162635803223"] <= 7)]).shape[0]

# Measurement - Filter_csv
def filter_time(dis):
    start_time = timeit.default_timer()
    dataframe = filter_and_select(dis)
    end_time = timeit.default_timer()
    return end_time - start_time

def measure_average_filter_time(files, cycles=10):
    avg_times = []
    for file_path in files:
        local_df = pd.read_csv(file_path)
        timings_pandas = [filter_time(local_df) for _ in range(cycles)]
        avg_time_pandas = average_time(timings_pandas)
        avg_times.append(avg_time_pandas)
    return avg_times

def run_filter(addresses):
    avg_times = measure_average_filter_time(addresses)

    for i, avg_time in enumerate(avg_times, 1):
        print(f"Pandas - Average filter time for data-{i}: {avg_time:.5f} seconds")
        


In [19]:
#run_filter(file_paths)

In [18]:
#run_filter(bigger_files)